In [6]:
import numpy as np
import pandas as pd
import math as m
import cmath
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
from sympy import pprint
# from scipy.optimize import minimize as GDlib
from scipy.optimize import minimize 
import MLE_functions as fn
from scipy.optimize import fsolve
from scipy.optimize import least_squares

In [7]:
pr_num = [sym.lambdify((fn.theta1, fn.theta2, fn.phi1, fn.phi2, fn.alpha), i) for i in fn.pr]   # lambdify the probabilities

# Improving the previous loss function, new code: 
def L(vars, params_tru = [0, m.pi/2, 0, 0, m.pi/4], nj_unnorm = None, lossfn = 'old',  N = 10000):
    if nj_unnorm != None:
        n_unnorm = nj_unnorm
    else:
        num_expt = fn.num_experiment(params= params_tru, N = N)                             # for getting the collapse count
        n_unnorm = num_expt[4]

    n = [i/sum(n_unnorm) for i in n_unnorm]                                                # normalise the counts
    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]
    
    lnl_p = [i * np.log(p) if i != 0 else 0 for i, p in zip(n, eq_pr)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.
    lnl_n = [i * np.log(i) if i != 0 else 0 for i in n]                         
    del_lnl = [(k-l) for k,l in zip(lnl_p, lnl_n)]
    loss = sum([abs(i) for i in del_lnl])

    if lossfn == 'old':
        loss = sum([abs(i) for i in lnl_p])

    return loss


def L_cross_entropy(p, n):
    lnl_p = [i * np.log(p) if p != 0 else 0 for i, p in zip(n, p)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.
    loss = sum([abs(i) for i in lnl_p])
    return loss


In [3]:
# one saved result for params_tru1 = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
"""tru =  [0.5235987755982988, 2.0943951023931953, 0.7853981633974483, 1.5707963267948966, 0.5235987755982988]  # same as [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
nj_unnorm =  [654, 558, 1169, 967, 2313, 521, 1260, 1152, 1406]
opt_inv =  [0.5254370610538157, 1.030984356143689, 0.7670544953445808, -1.6068690955721805, (0.5346630644404312+2.5302074683784063e-17j)]
opt_GD =  [0.5320294066145164, 1.0327066070674442, 0.7689496798867287, -1.6064174644839841, (0.5366920641304006+2.5302074683784063e-17j)]
fid_inv =  [0.9999333970821788, 0.9994887602573697]
fid_GD =  [0.9998777120836085, 0.9995482600825005]
ppm_inv =  [66.60291782123463, 511.2397426303428]
ppm_GD =  [122.28791639146141, 451.73991749947186]
"""

# for the usual params = [0, m.pi/2, 0, 0, m.pi/4]
"""nj_unnorm = [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]"""

'nj_unnorm = [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]'

In [10]:
#sample code for constrained minimization method
def constrain_p_sum(p):
    return (sum(p) - 1)

"""cons = {'type': 'eq', 'fun': constrain_p_sum}
init_guess = [1/9 for i in range(9)]
sol = minimize(L_cross_entropy, init_guess, method='SLSQP', constraints=cons)
print(sol)
"""


"cons = {'type': 'eq', 'fun': constrain_p_sum}\ninit_guess = [1/9 for i in range(9)]\nsol = minimize(L_cross_entropy, init_guess, method='SLSQP', constraints=cons)\nprint(sol)\n"

In [11]:
def residuals(vars, pr_list):
    pr_vars = [pr_num[i](*vars) for i in range(len(pr_num))]
    
    # pr_5 = [ pr_list[0]+ pr_list[1] , pr_list[2]+ pr_list[3], pr_list[4]+ pr_list[5], pr_list[6]+ pr_list[7], pr_list[8]]    # This is the given prob list # adding so to trick the least_squares function
    # pr_vars_5 = [pr_vars[0]+ pr_vars[1] , pr_vars[2]+ pr_vars[3], pr_vars[4]+ pr_vars[5], pr_vars[6]+ pr_vars[7], pr_vars[8]] # variables prob list, to be coverged to the given prob list 

    # res = [abs(i-j) for i,j in zip(pr_vars_5, pr_5)]
    
    return sum([abs(i-j) for i,j in zip(pr_vars, pr_list)])

    # return res
                   



def c_1_g(params):
    return (m.pi + 1e-12 - params[0])               # 1e-12 is added to avoid the floating point error

def c_2_g(params):
    return (m.pi + 1e-12 - params[1])

def c_3_g(params):
    return (m.pi + 1e-12 - params[2])

def c_4_g(params):
    return (m.pi + 1e-12 - params[3])

def c_5_g(params):
    return (m.pi/2 + 1e-12 - params[4])

def c_1_l(params):
    return (params[0])

def c_2_l(params):
    return (params[1]+ 1e-12)

def c_3_l(params):
    return (params[2]+ 1e-12)

def c_4_l(params):  
    return (params[3]+ 1e-12)

def c_5_l(params):
    return (params[4]+ 1e-12)


cons_ineq = ({'type': 'ineq', 'fun': c_1_g}, {'type': 'ineq', 'fun': c_2_g}, {'type': 'ineq', 'fun': c_3_g}, {'type': 'ineq', 'fun': c_4_g}, {'type': 'ineq', 'fun': c_5_g}, {'type': 'ineq', 'fun': c_1_l}, {'type': 'ineq', 'fun': c_2_l}, {'type': 'ineq', 'fun': c_3_l}, {'type': 'ineq', 'fun': c_4_l}, {'type': 'ineq', 'fun': c_5_l})


In [17]:
# trying to use the least_squares function and match the parameters to the given prob list
njn = [654, 558, 1169, 967, 2313, 521, 1260, 1152, 1406]
nju = [i/sum(njn) for i in njn]
# nju = [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]

# init_guess = [1/5 for i in range(5)]
init_guess = np.random.rand(5)
print(init_guess)
sol = minimize(residuals, init_guess, constraints = cons_ineq , method = 'SLSQP', args=(nju,) , tol = 1e-18, options = {'maxiter': 500})
print(sol)



[0.29846057 0.98846304 0.61744008 0.86249252 0.27381955]
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.01901622904666974
       x: [ 5.304e-01  2.086e+00  7.795e-01  1.441e+00  5.346e-01]
     nit: 295
     jac: [ 7.315e-02  4.963e-02  1.249e-02  7.914e-03  1.477e-01]
    nfev: 3509
    njev: 295


In [20]:
# tru1 = [0, m.pi/2, 0, 0, m.pi/4]
tru2  = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]

# in degrees
tru_deg = [i*180/m.pi for i in tru2]
sol_deg = [i*180/m.pi for i in sol.x]

print(tru_deg)
print(sol_deg)
#sol_deg : [4.386342135031111e-15, 89.72229793860824, 2.413763320647044, -5.752857106018894e-11, 45.70037272101446]

print(fn.fid(tru2, sol.x))

[29.999999999999996, 119.99999999999999, 45.0, 90.0, 29.999999999999996]
[30.392269651167027, 119.51965324492896, 44.6616841316508, 82.53580316398048, 30.631254046041548]
[0.9999465391983036, 0.9967832600054815]


In [42]:
pr_sol = [pr_num[i](*sol.x) for i in range(len(pr_num))]
print(nju)
print(pr_sol)

print(L_cross_entropy(nju, nju))
L_cross_entropy(pr_sol, nju)



[0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]
[0.08420014619721837, 0.08130021801802528, 0.1666586449593908, 0.08595557491630161, 0.0813002592565332, 0.16666466123984797, 0.08595557491630364, 0.08130025925653318, 0.16666466123984797]
2.1377889507324106


2.1383279430290285